In [1]:
import geopandas as gpd
import pandas as pd
import xarray as xr
import rioxarray
import xvec
import os

## Step 1)

First, we read in the MTBS Wildfire data files and package them in a list of dictionaries



In [9]:
mtbs_raster_data_dir = "./data/mtbs_rasters"
mtbs_perimeter_data_path = "./data/mtbs_perimeters/mtbs_perims_DD.shp"
mtbs_raster_files = os.listdir(mtbs_raster_data_dir)

mtbs_data = []

mtbs_gdf = gpd.read_file(mtbs_perimeter_data_path)

for file in mtbs_raster_files:
    data_struct = {}

    file_info = file[:-4].split("_")

    data_struct["state"] = file_info[1]
    data_struct["year"] = int(file_info[2])

    da_temp = rioxarray.open_rasterio(f"{mtbs_raster_data_dir}/{file}")
    da_temp = da_temp.rio.reproject("EPSG:4269")
    # We filter here to include only values of meaningful fire severity
    da_temp = da_temp.where((da_temp >= 2) & (da_temp < 5))
    data_struct["da"] = da_temp.copy()

    gdf_temp = (mtbs_gdf.loc[(mtbs_gdf["Incid_Type"]=="Wildfire") & (mtbs_gdf["Ig_Date"].dt.year == data_struct["year"]) & (mtbs_gdf["Event_ID"].str.startswith(data_struct["state"]))])
    gdf_temp = gdf_temp.set_index("Event_ID")[["geometry"]]
    gdf_temp = gdf_temp.to_crs(da_temp.spatial_ref.crs_wkt)
    data_struct["gdf"] = gdf_temp.copy()
    mtbs_data.append(data_struct)


In [21]:
calcd_dfs = []
da_name = "fire_severity"

for data in mtbs_data:

    da_temp = data["da"].xvec.zonal_stats(
        data["gdf"].geometry,
        x_coords="x",
        y_coords="y",
        stats=["mean", "max"],
        method="exactextract",
        index=True,
    )
    da_temp.name = da_name
    df_temp = da_temp.to_dataframe().reset_index()

    df_mean = df_temp.loc[df_temp["zonal_statistics"]=="mean"]
    df_mean = df_mean.rename(columns={da_name: da_name+"_mean"})

    df_max = df_temp.loc[df_temp["zonal_statistics"]=="max"]
    df_max = df_max.rename(columns={da_name: da_name+"_max"})


    df_temp =df_mean.merge(df_max, how="inner", on="Event_ID")[["Event_ID", da_name+"_mean", da_name+"_max"]]
    calcd_dfs.append(df_temp.copy())

fire_severity_df = pd.concat(calcd_dfs)
fire_severity_df = fire_severity_df.drop_duplicates(subset=["Event_ID"])
mtbs_gdf_final = mtbs_gdf.merge(fire_severity_df, how='left', on="Event_ID")

/Users/ericcollins/opt/miniconda3/envs/scenariomip/lib/python3.11/site-packages/exactextract/exact_extract.py:330: RuntimeWarning: Spatial reference system of input features does not exactly match raster.
  warnings.warn(
